In [28]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

import vvnet
__import__("importlib").reload(vvnet)
from vvnet import VVNet, INPUT_SHAPE

In [29]:
model = VVNet()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 128, 256, 256, 3)]   0         []                            
                                                                                                  
 sequential_95 (Sequential)  (None, 128, 256, 256, 64)    14144     ['input_7[0][0]']             
                                                                                                  
 sequential_96 (Sequential)  (None, 128, 256, 256, 64)    49280     ['sequential_95[0][0]']       
                                                                                                  
 max_pooling3d_18 (MaxPooli  (None, 64, 128, 128, 64)     0         ['sequential_96[0][0]']       
 ng3D)                                                                                      

In [30]:
var = np.random.rand(*INPUT_SHAPE)
model.predict(np.array([var]))

1/1 [==============================] - 39s 39s/step


array([[[[[0.44371125],
          [0.44371125],
          [0.44371125],
          ...,
          [0.44371125],
          [0.44371125],
          [0.44371125]],

         [[0.44371125],
          [0.44371125],
          [0.44371125],
          ...,
          [0.44371125],
          [0.44371125],
          [0.44371125]],

         [[0.44371125],
          [0.44371125],
          [0.44371125],
          ...,
          [0.44371125],
          [0.44371125],
          [0.44371125]],

         ...,

         [[0.44371125],
          [0.4467877 ],
          [0.44371125],
          ...,
          [0.44371125],
          [0.445011  ],
          [0.44371125]],

         [[0.44371125],
          [0.44371125],
          [0.44371125],
          ...,
          [0.44371125],
          [0.44595152],
          [0.44371125]],

         [[0.44371125],
          [0.44371125],
          [0.449417  ],
          ...,
          [0.44371125],
          [0.44716328],
          [0.44371125]]],


        [[[0.4437